# Bootstrap tools

> Bootstrap cells from seq cell dataframe

In [123]:
#| default_exp bootstrap_tools

In [70]:
#| hide
import nbdev; nbdev.nbdev_export()

# Import

In [1]:
# | export 

import pandas as pd
import numpy as np
from scipy.signal import correlate
import itertools

# Data Test

In [2]:
data = [('ACCA', 1, 1, 0, -1), ('ACCA', 2, 1, 1,-2), 
        ('CAAC', 1, 2, 0,-1), ('ACCA', 3, 4, 1,-1), 
        ('CAAC', 2, 2, 1,-2), ('CCCA', 3, 3, 1, -1)]
df = pd.DataFrame(data, columns=['seq', 'cb_encode', 'counts', 'sort_population', 'log_units_mnase'])
df

,seq,cb_encode,counts,sort_population,log_units_mnase
0,ACCA,1,1,0,-1
1,ACCA,2,1,1,-2
2,CAAC,1,2,0,-1
3,ACCA,3,4,1,-1
4,CAAC,2,2,1,-2
5,CCCA,3,3,1,-1


In [3]:
metadata = pd.DataFrame([[1,-1.0, 0], [2,-2.0, 1], [3,-1, 1] ], columns=['cb_encode', 'log_units_mnase', 'sort_population'])
metadata

,cb_encode,log_units_mnase,sort_population
0,1,-1.0,0
1,2,-2.0,1
2,3,-1.0,1


In [4]:
metadata2 = pd.DataFrame([[1,-1.0, 0], [2,-2.0, 1], [3,-1, 1], [4, -1.0, 0]], columns=['cb_encode', 'log_units_mnase', 'sort_population'])
data = [('ACCA', 1, 1, 0, -1), ('ACCA', 2, 1, 1,-2), 
        ('CAAC', 1, 2, 0,-1), ('ACCA', 3, 4, 1,-1), 
        ('CAAC', 2, 2, 1,-2), ('CCCA', 3, 3, 1, -1), ('ACCA', 4, 4, 0,-1)]
df2 = pd.DataFrame(data, columns=['seq', 'cb_encode', 'counts', 'sort_population', 'log_units_mnase'])

# Functions

In [16]:
# | export 

def sample_cells(df, metadata, n_sample):
    selected_cells = np.random.choice(metadata['cb_encode'], size=n_sample)
    subsampled_df = df.set_index('cb_encode').loc[selected_cells,:].reset_index()
    subsampled_meta = metadata.set_index('cb_encode').loc[selected_cells,:]
    return selected_cells, subsampled_meta, subsampled_df


In [17]:
cells, meta, b = sample_cells(df, metadata, 2)
assert len(b['cb_encode'].unique()) <= 2
assert len(cells) == 2

In [8]:
# | export

def fill_not_found_sequences(df, metadata):
    comb_seq_cb = pd.MultiIndex.from_tuples(itertools.product(df['seq'].unique(), metadata['cb_encode']), names= ('seq', 'cb_encode'))
    filled = df.set_index(['seq', 'cb_encode']).reindex(comb_seq_cb)
    filled.reset_index(level=0, inplace=True)
    filled['counts'] = filled['counts'].fillna(0)
    filled = filled.fillna(metadata.set_index('cb_encode').to_dict()).reset_index()
    return filled

In [9]:
filled_df = fill_not_found_sequences(df, metadata)
filled_df.pivot(index='seq', columns='cb_encode', values='counts')

cb_encode,1,2,3
seq,,,
ACCA,1.0,1.0,4.0
CAAC,2.0,2.0,0.0
CCCA,0.0,0.0,3.0


In [10]:
# | export

def digestion_profile_matrix(df, aggregation='mean'):
    averages = df.groupby(['seq','log_units_mnase']).aggregate(aggregation).reset_index()
    averages = averages.sort_values('log_units_mnase').reset_index(drop=True)
    dig_matrix = averages.pivot(index='seq', columns='log_units_mnase', values='counts').fillna(0)
    return dig_matrix


In [11]:
dig = digestion_profile_matrix(filled_df)
assert (dig.values == np.array([[1.0,2.5],[2.0,1.0],[0.0,1.5]])).all()

In [12]:
dig

log_units_mnase,-2.0,-1.0
seq,,
ACCA,1.0,2.5
CAAC,2.0,1.0
CCCA,0.0,1.5


In [18]:
# | export
def fast_digestion_profile(df, metadata):
    """ create digestion profile without filling 0 in the original dataframe.
    This is done by summing the counts per seq and log_units_mnase and divide by the number of
    cells per mnase"""
    count_cell_per_mnase = metadata.groupby('log_units_mnase')['log_units_mnase'].count()


    sum_counts = df.groupby(['seq','log_units_mnase'])['counts'].sum()
    multi_df2 = df.set_index(['seq','log_units_mnase'])
    avg_count = (sum_counts / count_cell_per_mnase).reset_index()

    multi_df2['avg_count'] = (sum_counts / count_cell_per_mnase)
    dig_matrix = multi_df2.groupby(level=[0,1])['avg_count'].mean().to_frame().unstack().fillna(0).loc[:, 'avg_count']
    return dig_matrix


In [22]:
fast_dig = fast_digestion_profile(df, metadata)
assert (fast_dig.values == np.array([[1.0,2.5],[2.0,1.0],[0.0,1.5]])).all()

In [23]:
# | export 

def bootstrap_population_filling(seqcell, pop, samples_size, metadata):
    all_g0 = seqcell.loc[seqcell['sort_population']==pop]
    sampled, g0_boot = sample_cells(all_g0, samples_size)
    filled_df = fill_not_found_sequences(g0_boot, metadata)
    g0_digprof = digestion_profile_matrix(g0_boot)
    return g0_digprof

In [28]:
# | export 

def bootstrap_population(seqcell, pop, samples_size, metadata):
    df_pop = seqcell.loc[seqcell['sort_population']==pop]
    metadata_pop = metadata.loc[metadata['sort_population']==pop]
    sampled, metadata, df = sample_cells(df_pop, metadata_pop, samples_size)
    digprof = fast_digestion_profile(df, metadata)
    return digprof

In [37]:

single_bootstrap = bootstrap_population(df, 1, 1000, metadata)
assert all(single_bootstrap.loc['ACCA'] ==[1.,4.])


In [40]:
# | export

def correlate_sequence(row, other, seq_name):
    try:
        other_row = other.loc[seq_name, :]
        cc = correlate(row, other_row, "full")
        return cc
    except KeyError:
        return np.array([np.nan] * other.shape[1]*2-1)

In [41]:
row = dig.loc['ACCA']
other = dig
c = correlate_sequence(row, other, 'CAAC')
c
assert c.max() == 5.
assert c.argmax() == 2

In [42]:
# | export

def cross_correlation_on_all_sequences(g0_profile, inter_profile):
    corr_df_shape = (g0_profile.shape[0], g0_profile.shape[1]*2-1)
    range_values = (-1*(g0_profile.shape[1]-1), g0_profile.shape[1] )
    corr_df = pd.DataFrame(np.zeros(corr_df_shape), index=g0_profile.index, columns=list(range(*range_values)))

    for nm,row in g0_profile.iterrows():
        corr = correlate_sequence(row, inter_profile, nm)
        corr_df.loc[nm, :] = corr
    corr_df = corr_df.dropna()
    return corr_df


In [44]:
corr_df = cross_correlation_on_all_sequences(dig, dig)
assert (corr_df.loc[:, -1] == corr_df.loc[:, 1]).all()

In [45]:
corr_df

,-1,0,1
seq,,,
ACCA,2.5,7.25,2.5
CAAC,2.0,5.00,2.0
CCCA,0.0,2.25,0.0


In [53]:
# | export 

def gmean_filter(pop1_digmatrix, pop2_digmatrix, threshold):
    gm_score = (pop1_digmatrix.aggregate('mean', axis=1) * pop2_digmatrix.aggregate('mean', axis=1)).dropna()
    i = gm_score.index[gm_score>threshold]
    return gm_score, i
    

In [55]:
pop1_digmatrix = pd.DataFrame([['AAA', 10, 1, 1],
                              ['BBB', 1, 0, 0], 
                              ['CCC', 1,1,10]], columns=['seq', -1, 0, 1]).set_index('seq')
pop2_digmatrix = pd.DataFrame([['AAA', 5, 1, 0],
                              ['BBB', 1, 1, 1], 
                              ['DDD', 1,1,10]], columns=['seq', -1, 0, 1]).set_index('seq')
gm_score, idx = gmean_filter(pop1_digmatrix, pop2_digmatrix, threshold=1)
assert idx == 'AAA'

## Main bootstrap function

In [56]:
# | export 

def single_iteration_bootstrap_cc(seqcell, populations, samples_bootstrap, metadata, threshold=1):
    g0_digprof = bootstrap_population(seqcell, populations[0], samples_bootstrap, metadata)
    inter_digprof = bootstrap_population(seqcell, populations[1], samples_bootstrap, metadata)
    # filter low mean counts
    gmean, idx = gmean_filter(g0_digprof, inter_digprof, threshold=threshold)
    g0_digprof = g0_digprof.loc[idx, :]
    inter_digprof = inter_digprof.loc[idx, :]
    
    cc = cross_correlation_on_all_sequences(g0_digprof, inter_digprof)
    return cc


In [57]:
df

,seq,cb_encode,counts,sort_population,log_units_mnase
0,ACCA,1,1,0,-1
1,ACCA,2,1,1,-2
2,CAAC,1,2,0,-1
3,ACCA,3,4,1,-1
4,CAAC,2,2,1,-2
5,CCCA,3,3,1,-1


In [58]:
b = single_iteration_bootstrap_cc(df, [1,1],100, metadata, threshold=0)
assert (b.loc[:, -1] == b.loc[:, 1]).all()

In [59]:
# | export 

def iter_bootstrap_cc(seqcell, metadata, populations= [0,1], samples_bootstrap=200, n_iter=100, threshold=1):    
    cross_correlations = []

    for _ in range(n_iter):
        cc = single_iteration_bootstrap_cc(seqcell, populations, samples_bootstrap, metadata, threshold=threshold)
        cross_correlations.append(cc)
        
    return cross_correlations


In [72]:
b1 = iter_bootstrap_cc(df, metadata, [1,1], 100, 1, threshold=0 )

In [60]:
# | export 

def average_results_bootstrap(cross_correlations):
    cat_correlations = pd.concat(cross_correlations, axis=0, join='inner')
    mean_cc = cat_correlations.groupby(cat_correlations.index).mean()
    se_cc = cat_correlations.groupby(cat_correlations.index).sem()
    max_cc = mean_cc.idxmax(axis = 1)
    return mean_cc, se_cc, max_cc

In [67]:
corr_df2 = corr_df.copy()
corr_df2.loc[:,1] = [7.5,2.,4.]
cross_correlations = [corr_df, corr_df2]
mean_cc, se_cc, max_cc = average_results_bootstrap(cross_correlations)
assert (mean_cc.loc['ACCA', :] == np.array([2.5,7.25,5])).all()
assert (max_cc == 0).all()

In [68]:
# | export

def aggregate_results_bootstrap(cross_correlations, aggregations=['mean', 'var']):
    cat_correlations = pd.concat(cross_correlations, axis=0, join='inner')
    agg_cc = cat_correlations.groupby(cat_correlations.index).aggregate(aggregations)
    return agg_cc

In [69]:
aggregs = aggregate_results_bootstrap(cross_correlations)
assert aggregs.loc['ACCA', 1]['var'] == (7.5-5)**2 + (2.5-5)**2